In [1]:
import geopandas as gpd
import os

In [4]:
path = r'T:\MPO\RTP\FY20 2045 Update\Data and Resources\Data\GISData\Updated'
datapath = r'T:\MPO\RTP\FY20 2045 Update\Data and Resources\Data\ForAppendixF'

In [2]:
def RTP_counted_by_intersection(shapefile='Roadway_lines',
                                folder='Historic',
                                file='NationalRegisterHistoricSitesCLMPO.shp',
                                transit=False, ftn=False, returnID=False):
    if transit:
        rtp = gpd.read_file(os.path.join(datapath, 'RTP', shapefile+'.shp'))
    else:
        rtp = gpd.read_file(os.path.join(path, 'Constrained_'+shapefile+'.shp'))
    # 100 feet buffer
    rtp['buffered'] = rtp.buffer(100)
    rtp = rtp.set_geometry('buffered')
    rtp = rtp.to_crs(epsg=3857)
    env = gpd.read_file(os.path.join(datapath, folder, file))
    if env.crs != 'EPSG:3857':
        env = env.to_crs(epsg=3857)
    joined = gpd.tools.sjoin(rtp, env, how="inner")
    if transit:
        if ftn:
            joined = joined.drop_duplicates(subset=['route', 'geometry'])
            res = joined[['route', 'geometry']].groupby(['route']).agg('count')
        elif 'name_left' in joined.columns:
            joined = joined.drop_duplicates(subset=['name_left', 'geometry'])
            res = joined[['name_left', 'geometry']].groupby(['name_left']).agg('count')
        else:
            joined = joined.drop_duplicates(subset=['name', 'geometry'])
            res = joined[['name', 'geometry']].groupby(['name']).agg('count')
    else:
        joined = joined.drop_duplicates(subset=['RTP_ID', 'geometry'])
        if returnID:
            res = joined[['Category', 'RTP_ID']].groupby(['Category']).agg(lambda x: set(x))
        else:
            res = joined[['Category', 'geometry']].groupby(['Category']).agg('count')
    return res

In [10]:
df1=RTP_counted_by_intersection(shapefile='Roadway_lines', folder='WaterQuality', file='wetlandsCLMPO.shp')

In [11]:
df2=RTP_counted_by_intersection(shapefile='Roadway_points', folder='WaterQuality', file='wetlandsCLMPO.shp')

In [18]:
df1

,geometry
Category,
Added Freeway Lanes or Major Interchange Improvements,2
Arterial Capacity Improvements,7
New Arterial Link or Interchange,1
New Collectors,14
Study,9
Urban Standards,19


In [15]:
list(df1.index)

['Added Freeway Lanes or Major Interchange Improvements',
 'Arterial Capacity Improvements',
 'New Arterial Link or Interchange',
 'New Collectors',
 'Study',
 'Urban Standards']

In [13]:
df2

,geometry
Category,
Arterial Capacity Improvements,2


In [16]:
list(df2.index)

['Arterial Capacity Improvements']

In [17]:
for ind in list(df2.index):
    df1.loc[ind, 'geometry'] = df1.loc[ind, 'geometry'] + df2.loc[ind, 'geometry']

In [19]:
df2.shape[0] != 0

True

In [21]:
def combine_RTP_by_mode(mode='roadway', folder='Historic',
                                file='NationalRegisterHistoricSitesCLMPO.shp'):
    if mode == 'roadway':
        df1=RTP_counted_by_intersection(shapefile='Roadway_lines', folder=folder, file=file)
        df2=RTP_counted_by_intersection(shapefile='Roadway_points', folder=folder, file=file)
    elif mode == 'bikeped':
        df1=RTP_counted_by_intersection(shapefile='BikePed', folder=folder, file=file)
        df2=RTP_counted_by_intersection(shapefile='BikePed_points', folder=folder, file=file)
        
    if df2.shape[0] != 0:
        for ind in list(df2.index):
            df1.loc[ind, 'geometry'] = df1.loc[ind, 'geometry'] + df2.loc[ind, 'geometry']
    return df1

In [22]:
combine_RTP_by_mode()

,geometry
Category,
Study,4
Transit Oriented Development Implementation,1
Urban Standards,1


In [26]:
categories = ['Auto']*7 + ['Transit']*2 + ['Bike/Ped']*4

project_types = ['Added Freeway Lanes or Major Interchange Improvements',
                 'Arterial Capacity Improvements',
                 'New Arterial Link or Interchange',
                 'New Collectors',
                 'Transit Oriented Development Implementation',
                 'Study',
                 'Urban Standards',
                 'Frequent Transit Network',
                 'Stations',
                 'Multi-Use Paths without Road Project',
                 'Multi-Use Paths with Road Project',
                 'On-Street Lanes or Routes with Road Project',
                 'On-Street Lanes or Routes without Road Project']

In [27]:
import pandas as pd

In [29]:
import numpy as np

In [31]:
len(categories)

13

In [32]:
len(project_types)

13

In [37]:
df = pd.DataFrame(data={'Project Category': categories, 'Project Type': project_types})

In [38]:
folder='Historic'
file='NationalRegisterHistoricSitesCLMPO.shp'

In [39]:
'HistoricSites' in file

True

In [40]:
df['Historic Sites'] = 0

In [41]:
df

,Project Category,Project Type,Historic Sites
0,Auto,Added Freeway Lanes or Major Interchange Impro...,0
1,Auto,Arterial Capacity Improvements,0
2,Auto,New Arterial Link or Interchange,0
3,Auto,New Collectors,0
4,Auto,Transit Oriented Development Implementation,0
5,Auto,Study,0
6,Auto,Urban Standards,0
7,Transit,Frequent Transit Network,0
8,Transit,Stations,0
9,Bike/Ped,Multi-Use Paths without Road Project,0


In [42]:
rwdf = combine_RTP_by_mode(mode='roadway')

In [43]:
rwdf.shape[0]

,geometry
Category,
Study,4
Transit Oriented Development Implementation,1
Urban Standards,1


In [44]:
var = 'Historic Sites'

In [48]:
rwdf.loc['Study', 'geometry']

4

In [49]:
df.loc[df['Project Type'] == 'Study', var] = rwdf.loc['Study', 'geometry']

In [50]:
bpdf = combine_RTP_by_mode(mode='bikeped')

In [51]:
bpdf

,geometry
Category,
On-Street Lanes or Routes With Road Project,1
On-Street Lanes or Routes Without Road Project,6


In [52]:
def combine_RTP_for_each_env_factor(folder='Historic',
                                    file='NationalRegisterHistoricSitesCLMPO.shp'):
    df = pd.DataFrame(data={'Project Category': categories, 'Project Type': project_types})
    if 'HistoricSites' in file:
        var = 'Historic Sites'
    df[var] = 0
    # rwdf: roadway dataframe
    rwdf = combine_RTP_by_mode(mode='roadway', folder=folder, file=file)
    if rwdf.shape[0] != 0:
        for ind in list(rwdf.index):
            df.loc[df['Project Type'] == ind, var] = rwdf.loc[ind, 'geometry']
    # bpdf: bike/ped dataframe
    bpdf = combine_RTP_by_mode(mode='bikeped', folder=folder, file=file)
    if bpdf.shape[0] != 0:
        for ind in list(bpdf.index):
            df.loc[df['Project Type'] == ind, var] = bpdf.loc[ind, 'geometry']
    # ftndf: frequent transit network dataframe
    ftndf = RTP_counted_by_intersection(shapefile='FrequentTransitNetwork', 
                                        folder=folder, file=file,
                                        transit=True, ftn=True)
    df.loc[df['Project Type'] == 'Frequent Transit Network', var] = ftndf.shape[0]
    # stadf: stations dataframe
    stadf = RTP_counted_by_intersection(shapefile='stations',
                                        folder=folder, file=file, transit=True)
    df.loc[df['Project Type'] == 'Stations', var] = stadf.shape[0]
    return df

In [53]:
combine_RTP_for_each_env_factor()

,Project Category,Project Type,Historic Sites
0,Auto,Added Freeway Lanes or Major Interchange Impro...,0
1,Auto,Arterial Capacity Improvements,0
2,Auto,New Arterial Link or Interchange,0
3,Auto,New Collectors,0
4,Auto,Transit Oriented Development Implementation,1
5,Auto,Study,4
6,Auto,Urban Standards,1
7,Transit,Frequent Transit Network,24
8,Transit,Stations,1
9,Bike/Ped,Multi-Use Paths without Road Project,0
